In [1]:
#Importing all the needful libraries

import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize

In [2]:
#Reading all the json files
users = pd.read_json("users.json", lines = True)
brands = pd.read_json("brands.json", lines = True)
receipts = pd.read_json('receipts.json', lines = True)
receipts.rename(columns={'_id': 'receiptId'}, inplace=True)

<h3> DATA CLEANING AND PREPROCESSING </h3>

To convert timestamp to datetime datetime.fromtimestamp() method could be used easily but this method could not deal with NULL values therefore a custom function is created to handle that.

In [3]:
# CREATED A FUNCTION TO CONVERT TIMESTAMP TO DATETIME
from datetime import datetime
def to_dt(t):
    dt_object = []
    timestamp = list(t/1000)
    for i in timestamp:
        if pd.isna(i):
            dt_object.append(i)
        else:
            d = datetime.fromtimestamp(i)
            dt_object.append(d)
 
    return dt_object

In [4]:
#USERS DATA PREPROCESSING

users['_id'] = pd.json_normalize(users['_id'])
users['createdDate'] = pd.json_normalize(users['createdDate'])
users['lastLogin'] = pd.json_normalize(users['lastLogin'])
users.createdDate = to_dt(users.createdDate)
users.lastLogin = to_dt(users.lastLogin)

In [5]:
# BRANDS DATA PREPROCESSING

cpg = pd.DataFrame(pd.json_normalize(brands['cpg']))
brands.drop('cpg', inplace = True, axis = 1)
brands['cpg_ref'] = cpg['$ref']
brands['cpg_id'] = cpg['$id.$oid']
brands['_id'] = pd.json_normalize(brands['_id'])

In [6]:
brands.head()

,_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg_ref,cpg_id
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7


In [7]:
#RECEIPTS DATA PREPROCESSING

receipts['receiptId'] = pd.json_normalize(receipts['receiptId'])
receipts['createDate'] = pd.json_normalize(receipts['createDate'])
receipts['dateScanned'] = pd.json_normalize(receipts['dateScanned'])
receipts['finishedDate'] = pd.json_normalize(receipts['finishedDate'])
receipts['modifyDate'] = pd.json_normalize(receipts['modifyDate'])
receipts['pointsAwardedDate'] = pd.json_normalize(receipts['pointsAwardedDate'])
receipts['purchaseDate'] = pd.json_normalize(receipts['purchaseDate'])
receipts.createDate = to_dt(receipts.createDate)
receipts.dateScanned = to_dt(receipts.dateScanned)
receipts.finishedDate = to_dt(receipts.finishedDate)
receipts.modifyDate = to_dt(receipts.modifyDate)
receipts.pointsAwardedDate = to_dt(receipts.pointsAwardedDate)
receipts.purchaseDate = to_dt(receipts.purchaseDate)

In [8]:
#CREATING NEW TABLE FOR LIST OF ITEMS IN EACH RECEIPT

receipts_item_list = receipts[['rewardsReceiptItemList','receiptId']].copy()
receipts.drop('rewardsReceiptItemList', inplace = True, axis = 1)

In [9]:
receipts_item_list.head()

,rewardsReceiptItemList,receiptId
0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",5ff1e1eb0a720f0523000575
1,"[{'barcode': '4011', 'description': 'ITEM NOT ...",5ff1e1bb0a720f052300056b
2,"[{'needsFetchReview': False, 'partnerItemId': ...",5ff1e1f10a720f052300057a
3,"[{'barcode': '4011', 'description': 'ITEM NOT ...",5ff1e1ee0a7214ada100056f
4,"[{'barcode': '4011', 'description': 'ITEM NOT ...",5ff1e1d20a7214ada1000561


In [10]:
#EXPLODING THE LIST OF DICTIONARY TO INDIVIDUAL DICTIONARY TO CAPTURE INFORMATION OF EACH ITEM

receipts_item_list = receipts_item_list.explode('rewardsReceiptItemList')

In [11]:
receipts_item_list.head()

,rewardsReceiptItemList,receiptId
0,"{'barcode': '4011', 'description': 'ITEM NOT F...",5ff1e1eb0a720f0523000575
1,"{'barcode': '4011', 'description': 'ITEM NOT F...",5ff1e1bb0a720f052300056b
1,"{'barcode': '028400642255', 'description': 'DO...",5ff1e1bb0a720f052300056b
2,"{'needsFetchReview': False, 'partnerItemId': '...",5ff1e1f10a720f052300057a
3,"{'barcode': '4011', 'description': 'ITEM NOT F...",5ff1e1ee0a7214ada100056f


In [12]:
#NORMALIZING ALL THE DICTIONARIES INTO A TABLE TO QUERY OVER

normalized_data = {}

# Find all unique keys across all dictionaries
all_keys = set()
for index, row in receipts_item_list.iterrows():
    row_data = row['rewardsReceiptItemList']
    if isinstance(row_data, dict):
        all_keys.update(row_data.keys())

# Iterate through each row in the original DataFrame
for index, row in receipts_item_list.iterrows():
    row_data = row['rewardsReceiptItemList']
    row_id = row['receiptId']
    
    # Check if the row_data is a dictionary (skip NaN values)
    if isinstance(row_data, dict):
        if 'receiptId' not in normalized_data:
            normalized_data['receiptId'] = []
        normalized_data['receiptId'].append(row_id)
        
        # Iterate through all keys
        for key in all_keys:
            value = row_data.get(key, None)  # Use None if key is missing
            # Create a new key in the normalized_data dictionary if it doesn't exist
            if key not in normalized_data:
                normalized_data[key] = []
            
            # Append the value to the corresponding key's list
            normalized_data[key].append(value)
    else:
        for key in all_keys:
            normalized_data[key].append(None)
        if 'receiptId' not in normalized_data:
            normalized_data['receiptId'] = []
        normalized_data['receiptId'].append(row_id)

# Create a new DataFrame from the normalized_data dictionary
new_df = pd.DataFrame(normalized_data)
print(new_df.shape)

(7381, 35)


In [13]:
receipts_item_list = new_df

In [14]:
new_df.columns

Index(['receiptId', 'userFlaggedQuantity', 'userFlaggedDescription',
       'discountedItemPrice', 'originalFinalPrice', 'originalMetaBriteBarcode',
       'originalMetaBriteDescription', 'itemNumber', 'targetPrice',
       'quantityPurchased', 'rewardsGroup', 'needsFetchReviewReason',
       'deleted', 'partnerItemId', 'originalMetaBriteQuantityPurchased',
       'metabriteCampaignId', 'barcode', 'competitiveProduct',
       'rewardsProductPartnerId', 'finalPrice', 'originalReceiptItemText',
       'pointsNotAwardedReason', 'userFlaggedPrice', 'description',
       'brandCode', 'priceAfterCoupon', 'userFlaggedNewItem', 'pointsEarned',
       'competitorRewardsGroup', 'originalMetaBriteItemPrice', 'itemPrice',
       'preventTargetGapPoints', 'userFlaggedBarcode', 'needsFetchReview',
       'pointsPayerId'],
      dtype='object')

In [15]:
#CREATING A BRIDGE TABLE BETWEEN USERS AND RECEIPTS TO REDUCE DATA REDUNDACNY
users_receipts = receipts[['receiptId', 'userId']].copy()

In [16]:
#CREATING A BRIDGE TABLE BETWEEN CATEGORY AND CATEGORY_CODE TO REDUCE DATA REDUNDACNY
category_code = brands[['category', 'categoryCode']].copy()

In [17]:
category_code.head()

,category,categoryCode
0,Baking,BAKING
1,Beverages,BEVERAGES
2,Baking,BAKING
3,Baking,BAKING
4,Candy & Sweets,CANDY_AND_SWEETS


In [18]:
#DROPPING THE COLUMNS WHICH ARE TRANFERRED TO NEW DATAFRAME
brands.drop('categoryCode', inplace = True, axis = 1)
receipts.drop('userId', inplace = True, axis = 1)

<h3> DATA QUALITY CHECK AND ANALYSIS </h3>

In [19]:
# CHECKING DUPLICATE DATA

print("The number of duplicate rows in Users table: ",users.duplicated().sum())
print("The number of duplicate rows in Brands table: ",brands.duplicated().sum())
print("The number of duplicate rows in Receipts table: ",receipts.duplicated().sum())
print("The number of duplicate rows in receipts_item_list table: ",receipts_item_list.duplicated().sum())
print("The number of duplicate rows in category_code table: ",category_code.duplicated().sum())
print("The number of duplicate rows in users_receipts table: ",users_receipts.duplicated().sum())

The number of duplicate rows in Users table:  283
The number of duplicate rows in Brands table:  0
The number of duplicate rows in Receipts table:  0
The number of duplicate rows in receipts_item_list table:  0
The number of duplicate rows in category_code table:  1134
The number of duplicate rows in users_receipts table:  0


In [20]:
# DROPPING ALL THE DUPLICATE DATA FROM THE TABLES
users.drop_duplicates(inplace = True)
category_code.drop_duplicates(inplace = True)

In [21]:
# COUNTING NULL VALUES IN ALL THE TABLEs

print("The number of null values in users table in column-wise format: \n",users.isna().sum())

The number of null values in users table in column-wise format: 
 _id              0
active           0
createdDate      0
lastLogin       40
role             0
signUpSource     5
state            6
dtype: int64


In [22]:
print("The number of null values in brands table in column-wise format: \n",brands.isna().sum())

The number of null values in brands table in column-wise format: 
 _id            0
barcode        0
category     155
name           0
topBrand     612
brandCode    234
cpg_ref        0
cpg_id         0
dtype: int64


In [23]:
print("The number of null values in receipts table in column-wise format: \n",receipts.isna().sum())

The number of null values in receipts table in column-wise format: 
 receiptId                    0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptStatus         0
totalSpent                 435
dtype: int64


In [24]:
print("The number of null values in receipts_item_list table in column-wise format: \n",receipts_item_list.isna().sum())

The number of null values in receipts_item_list table in column-wise format: 
 receiptId                                0
userFlaggedQuantity                   7082
userFlaggedDescription                7176
discountedItemPrice                   1612
originalFinalPrice                    7372
originalMetaBriteBarcode              7310
originalMetaBriteDescription          7371
itemNumber                            7228
targetPrice                           7003
quantityPurchased                      614
rewardsGroup                          5650
needsFetchReviewReason                7162
deleted                               7372
partnerItemId                          440
originalMetaBriteQuantityPurchased    7366
metabriteCampaignId                   6518
barcode                               4291
competitiveProduct                    6736
rewardsProductPartnerId               5112
finalPrice                             614
originalReceiptItemText               1621
pointsNotAwardedRe

For this table we have 35 columns and all of them has missing values but some of them has very large amount of missing values, therefore we will drop all the columns with missing values more than 1000 except for brandcode as we will need it in answering the predetermined business questions.

In [25]:
missing_threshold = 1000

# Calculate the count of missing values in each column
missing_counts = receipts_item_list.isna().sum()

# Get the column names that meet the condition
columns_to_drop = missing_counts[missing_counts >= missing_threshold].index

# Drop columns that meet the condition
receipts_item_list_filtered = receipts_item_list.drop(columns=columns_to_drop)

print(receipts_item_list_filtered.shape)

(7381, 6)


In [26]:
receipts_item_list_filtered['brandCode'] = receipts_item_list["brandCode"]

In [27]:
receipts_item_list_filtered.head()

,receiptId,quantityPurchased,partnerItemId,finalPrice,description,itemPrice,brandCode
0,5ff1e1eb0a720f0523000575,5.0,1,26.00,ITEM NOT FOUND,26.00,None
1,5ff1e1bb0a720f052300056b,1.0,1,1,ITEM NOT FOUND,1,None
2,5ff1e1bb0a720f052300056b,1.0,2,10.00,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,None
3,5ff1e1f10a720f052300057a,NaN,1,None,None,None,None
4,5ff1e1ee0a7214ada100056f,4.0,1,28.00,ITEM NOT FOUND,28.00,None


In [28]:
receipts_item_list = receipts_item_list_filtered
receipts_item_list.head()

,receiptId,quantityPurchased,partnerItemId,finalPrice,description,itemPrice,brandCode
0,5ff1e1eb0a720f0523000575,5.0,1,26.00,ITEM NOT FOUND,26.00,None
1,5ff1e1bb0a720f052300056b,1.0,1,1,ITEM NOT FOUND,1,None
2,5ff1e1bb0a720f052300056b,1.0,2,10.00,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,None
3,5ff1e1f10a720f052300057a,NaN,1,None,None,None,None
4,5ff1e1ee0a7214ada100056f,4.0,1,28.00,ITEM NOT FOUND,28.00,None


In [29]:
print("The number of null values in category_code table in column-wise format: \n",category_code.isna().sum())

The number of null values in category_code table in column-wise format: 
 category         1
categoryCode    19
dtype: int64


In [30]:
# COUNTING NULL VALUES IN ALL THE TABLEs

print("The number of null values in users_receipts table in column-wise format: \n",users_receipts.isna().sum())

The number of null values in users_receipts table in column-wise format: 
 receiptId    0
userId       0
dtype: int64


In [31]:
# ANALYZING CATEGORY_CODE DATA

category_code

,category,categoryCode
0,Baking,BAKING
1,Beverages,BEVERAGES
4,Candy & Sweets,CANDY_AND_SWEETS
7,Condiments & Sauces,NaN
8,Canned Goods & Soups,NaN
9,Baking,NaN
11,NaN,NaN
13,Magazines,NaN
14,Breakfast & Cereal,NaN
15,Beer Wine Spirits,NaN


As in the above dataframe, it can be seen that for category: bakery, the category_code is BAKERY as well as NaN which does not make sense. This is the case for other categories as well, therefore only relevant values are kept for category_code column. Another quality issue is in categories, for example one category is beauty and other category is beauty and personal care so there is a hint of ambiguity whether they are same or different, same is the case with bakery and bread & bakery.

In [32]:
# DROPPING CATEGORIES WITH TWO CATERY_CODE OUT OF WHICH ONE OF THEM IS "NaN"

category_code = category_code.drop_duplicates(subset='category', keep='first')

In [33]:
category_code

,category,categoryCode
0,Baking,BAKING
1,Beverages,BEVERAGES
4,Candy & Sweets,CANDY_AND_SWEETS
7,Condiments & Sauces,NaN
8,Canned Goods & Soups,NaN
11,NaN,NaN
13,Magazines,NaN
14,Breakfast & Cereal,NaN
15,Beer Wine Spirits,NaN
16,Health & Wellness,HEALTHY_AND_WELLNESS


In [34]:
# ANALYZING THE USER DATA

print("Number of users in users table: ",users._id.count())
print("Number of distinct users in users_receipts table: ",len(pd.unique(users_receipts['userId'])))

Number of users in users table:  212
Number of distinct users in users_receipts table:  258


The above values shows that there are some users whose data is missing from users table which is area of concern as users table should hold the information of all the users who are active on platform. In SQL, if we create userId in users table as primary key to maintain data integrity and refer the userId in users_receipts as foreign key from userId in users table it will violate the primary key property.

In [35]:
users.to_csv('users.csv', index = False)
brands.to_csv('brands.csv', index = False)
receipts.to_csv('receipts.csv', index = False)
users_receipts.to_csv('users_receipts.csv', index = False)
category_code.to_csv('category_code.csv', index = False)
receipts_item_list.to_csv('receipts_item_list.csv', index = False)